## MLFlow

In [4]:
import mlflow
import mlflow.sklearn

In [5]:
mlflow.__version__

'2.16.2'

In [6]:
!pip install scikit-learn   # scikit-learn 같은 경우 mlflow 의존성으로 함께 설치가 된다.

In [18]:
import pandas as pd
from sklearn.datasets import load_iris

iris = load_iris()
iris


# display(iris.data)   # feature data
# display(iris.target) # label data
# display(iris.feature_names) # col name

# df를 만들어거 모델 세팅하고 mlflow를 통해서 실험관리

df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['Lable'] = iris.target
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Lable
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [36]:
df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Lable
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state = 123)

In [33]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=0)

model.fit(X_train, y_train) # 모의고사: X_train, y_train

/Users/yijun/study/workspace/python/tmp_python/mlops/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=10)

In [34]:
pred = model.predict(X_test)

# X_test  # [[]] 두개인 이유는 행렬이기 때문이다.

In [35]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, pred)

0.9666666666666667

In [42]:
# 데이터 값의 표준 편차가 크다는걸 확인했다. -> scaler를 사용해서 값의 편차를 줄여줘야 겠따!

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_scaled = scaler.fit_transform(iris.data)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, iris.target, test_size=0.2, random_state=123)

model = LogisticRegression(max_iter=0)
model.fit(X_train, y_train)

pred = model.predict(X_test)
accuracy_score(y_test, pred)

/Users/yijun/study/workspace/python/tmp_python/mlops/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9333333333333333

In [44]:
df = pd.DataFrame(X_scaled, columns=iris.feature_names)
df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
count,1.500000e+02,1.500000e+02,1.500000e+02,1.500000e+02
mean,-1.468455e-15,-1.823726e-15,-1.610564e-15,-9.473903e-16
std,1.003350e+00,1.003350e+00,1.003350e+00,1.003350e+00
min,-1.870024e+00,-2.433947e+00,-1.567576e+00,-1.447076e+00
25%,-9.006812e-01,-5.923730e-01,-1.226552e+00,-1.183812e+00
50%,-5.250608e-02,-1.319795e-01,3.364776e-01,1.325097e-01
75%,6.745011e-01,5.586108e-01,7.627583e-01,7.906707e-01
max,2.492019e+00,3.090775e+00,1.785832e+00,1.712096e+00


#### 모델 실험 파라미터 및 모델 성능 기록

In [45]:
# ECS + M :마크다운
# ECS + Y :코드

mlflow.set_tracking_uri('http://127.0.0.1:5000')    # dev.upstage.com

print(mlflow.get_tracking_uri())

http://127.0.0.1:5000


In [56]:
exp = mlflow.set_experiment(experiment_name='iris_classifiacion')

exp.name
exp.experiment_id
exp.artifact_location
exp.creation_time   # 1726803122469

from datetime import datetime

datetime.fromtimestamp(exp.creation_time/1000)

datetime.datetime(2024, 9, 20, 12, 32, 2, 469000)

In [57]:
mlflow.autolog()

mlflow.start_run()  # start_run 이후 모델을 불러오고, 실험하는 코드들을 작성하면 된다.

model = LogisticRegression(max_iter=0)  # default=100
model.fit(X_train, y_train)
pred = model.predict(X_test)
accuracy = accuracy_score(y_test, pred)
print(f'정확도: {accuracy*100}')

mlflow.end_run()




2024/09/20 12:40:20 WARNING mlflow.utils.autologging_utils: MLflow sklearn autologging is known to be compatible with 0.24.1 <= scikit-learn <= 1.5.1, but the installed version is 1.5.2. If you encounter errors during autologging, try upgrading / downgrading scikit-learn to a compatible version, or try upgrading MLflow.
2024/09/20 12:40:25 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
/Users/yijun/study/workspace/python/tmp_python/mlops/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
Matplotlib is building t

정확도: 93.33333333333333


In [58]:
LogisticRegression? # 이 값들이 으 ui에 왼쪽에 나오는거다.

Init signature:
LogisticRegression(
    penalty='l2',
    *,
    dual=False,
    tol=0.0001,
    C=1.0,
    fit_intercept=True,
    intercept_scaling=1,
    class_weight=None,
    random_state=None,
    solver='lbfgs',
    max_iter=100,
    multi_class='deprecated',
    verbose=0,
    warm_start=False,
    n_jobs=None,
    l1_ratio=None,
)
Docstring:     
Logistic Regression (aka logit, MaxEnt) classifier.

In the multiclass case, the training algorithm uses the one-vs-rest (OvR)
scheme if the 'multi_class' option is set to 'ovr', and uses the
cross-entropy loss if the 'multi_class' option is set to 'multinomial'.
(Currently the 'multinomial' option is supported only by the 'lbfgs',
'sag', 'saga' and 'newton-cg' solvers.)

This class implements regularized logistic regression using the
'liblinear' library, 'newton-cg', 'sag', 'saga' and 'lbfgs' solvers. **Note
that regularization is applied by default**. It can handle both dense
and sparse input. Use C-ordered arrays or CSR matrices co

In [59]:
# 위 코드는 엄청 단순. 더 심플하게

mlflow.autolog()    # 알아서 필요한 데이터 기록해달라

with mlflow.start_run():
    model = LogisticRegression(max_iter=0)  # default=100
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, pred)
    print(f'정확도: {accuracy*100}')



2024/09/20 14:04:46 WARNING mlflow.utils.autologging_utils: MLflow sklearn autologging is known to be compatible with 0.24.1 <= scikit-learn <= 1.5.1, but the installed version is 1.5.2. If you encounter errors during autologging, try upgrading / downgrading scikit-learn to a compatible version, or try upgrading MLflow.
2024/09/20 14:04:46 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
/Users/yijun/study/workspace/python/tmp_python/mlops/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
2024/09/20 14:04:49 INFO

정확도: 93.33333333333333


In [62]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# 알고리즘별 성능 테스트

models = {
    "LogisticRegression": LogisticRegression(
        max_iter=0,
        C = 1.0,
        solver='lbfgs',
        random_state=123
    ), 
    "RandomForestClassifier": RandomForestClassifier(
        n_estimators=100,
        random_state=123
    ),
    "SVC": SVC(
        kernel='linear',
        random_state=123
    )
}



In [70]:

for model_name, model in models.items():
    # print(model_name, model)
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, pred)
    
    mlflow.log_param('log_param_key', 'log_param_value')
    mlflow.log_metric('log_metric_key', 0.01)

    print(f'모델명: {model_name}, 정확도: {accuracy*100}')
    

/Users/yijun/study/workspace/python/tmp_python/mlops/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


모델명: LogisticRegression, 정확도: 93.33333333333333
모델명: RandomForestClassifier, 정확도: 93.33333333333333


2024/09/20 14:18:33 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 53c515983d5f49f694bbe6c3c0deab23. Failed operations: [MlflowException("API request to http://127.0.0.1:5000/api/2.0/mlflow/runs/log-batch failed with exception HTTPConnectionPool(host=\'127.0.0.1\', port=5000): Max retries exceeded with url: /api/2.0/mlflow/runs/log-batch (Caused by ResponseError(\'too many 500 error responses\'))")]')]


모델명: SVC, 정확도: 93.33333333333333


In [ ]:
# 제일 좋은거만 출력하기
best_accuracy = 0

for model_name, model in models.items():
    # print(model_name, model)
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, pred)
    
    if accuracy > best_accuracy:
        print()
        
    mlflow.log_param('log_param_key', 'log_param_value')
    mlflow.log_metric('log_metric_key', 0.01)

    print(f'모델명: {model_name}, 정확도: {accuracy*100}')
    

In [75]:
# run name 설정해주기
from datetime import datetime
today = datetime.today().strftime('%Y-%M-%D')

# 이거 꼭 있어야하나?
mlflow.autolog()    

for model_name, model in models.items():
    
    with mlflow.start_run(run_name=f'{today} - {model_name}', nested=True): # nested는 새로 생성되는거 말고 안으로 생성되는거다.
        model.fit(X_train, y_train)
        pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, pred)
        
            
        mlflow.log_param('log_param_key', 'log_param_value')
        mlflow.log_metric('log_metric_key', 0.01)
        
        print(f'모델명: {model_name}, 정확도: {accuracy*100}')

/Users/yijun/study/workspace/python/tmp_python/mlops/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
2024/09/20 14:23:16 INFO mlflow.tracking._tracking_service.client: 🏃 View run 2024-23-09/20/24 - LogisticRegression at: http://127.0.0.1:5000/#/experiments/841097937802182006/runs/d6f9e3418b474df18f0f367787baeaad.
2024/09/20 14:23:16 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/841097937802182006.


모델명: LogisticRegression, 정확도: 93.33333333333333


2024/09/20 14:23:20 INFO mlflow.tracking._tracking_service.client: 🏃 View run 2024-23-09/20/24 - RandomForestClassifier at: http://127.0.0.1:5000/#/experiments/841097937802182006/runs/b4df364439944001a07482b97505bc3e.
2024/09/20 14:23:20 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/841097937802182006.


모델명: RandomForestClassifier, 정확도: 93.33333333333333


2024/09/20 14:23:23 INFO mlflow.tracking._tracking_service.client: 🏃 View run 2024-23-09/20/24 - SVC at: http://127.0.0.1:5000/#/experiments/841097937802182006/runs/d0944a5dda1b49d7b751f594c537f568.
2024/09/20 14:23:23 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/841097937802182006.


모델명: SVC, 정확도: 93.33333333333333


### MLFlow 모델 관리
- Stage 관리를 하겠다라는 뜻

4개의 함수
1. model_register()
2. promote_to_staging(): 모델 등록 & Staging 단계로 승격
3. promote_to_production(): Production 단계로 승격
4. archive_model(): 모델 아카이빙


In [94]:
# 원래는 stage 옵션으로 관리되는데 이게 deprecated되었따.
# 이렇게 tag 이용해서 하는건 최신 버전이다.

from mlflow.tracking import MlflowClient

client = MlflowClient()

# 모델 등록 함수 생성
def model_register(model_name, run_id):
    model_uri = f'runs:/{run_id}/model'
    model_version = mlflow.register_model(model_uri, model_name)
    return model_version    # workflow management

# staging 승격
def promote_to_staging(model_name, run_id):
    model_version = model_register(model_name, run_id)
    client.set_model_version_tag(
        name=model_name,
        version=model_version.version,
        key='stage',
        value='staging'
    )
    
    print(f'Model: {model_name}, Version: {model_version.version} is promoted to Staging..')
    
# production 승격
def promote_to_production(model_name, version):
    client.set_model_version_tag(
        name=model_name,
        version=version,
        key='stage',
        value='production'
    )
    
    print(f'Model: {model_name}, Version: {version} is promoted to Productions..')
   
# archive_model
def archive_model(model_name, version):
    client.set_model_version_tag(
        name=model_name,
        version=version,
        key='stage',
        value='archive'
    )
    
    print(f'Model: {model_name}, Version: {version} is moved to Archive..')
    


In [85]:
# 모델 등록
# http://127.0.0.1:5000/#/experiments/841097937802182006/runs/6b36de56cd8a441fbc9951f372971230

# 실험 id: 841097937802182006
# 실행 id: 6b36de56cd8a441fbc9951f372971230
run_id = "6b36de56cd8a441fbc9951f372971230"
model_name='RandomForestClassifier' # tags에서 가져옴
model_register(model_name, run_id)

Registered model 'RandomForestClassifier' already exists. Creating a new version of this model...
2024/09/20 15:21:19 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: RandomForestClassifier, version 3
Created version '3' of model 'RandomForestClassifier'.


<ModelVersion: aliases=[], creation_timestamp=1726813279073, current_stage='None', description='', last_updated_timestamp=1726813279073, name='RandomForestClassifier', run_id='6b36de56cd8a441fbc9951f372971230', run_link='', source='mlflow-artifacts:/841097937802182006/6b36de56cd8a441fbc9951f372971230/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='3'>

Experiments A
- model1
- model2

Experiments B
- model1
- model2

이런 구조다

In [90]:
# model 등록 및 Staging 승격
promote_to_staging(model_name, run_id)

Registered model 'RandomForestClassifier' already exists. Creating a new version of this model...
2024/09/20 15:22:59 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: RandomForestClassifier, version 6


Model: RandomForestClassifier, Version: 6 is promoted to Staging..


Created version '6' of model 'RandomForestClassifier'.


새로운 버전이 Production이 되면 그 이전 버전의 태그는 Archived로 "수동으로" 바꿔줘야한다.

In [99]:
promote_to_production(model_name, '6')


Model: RandomForestClassifier, Version: 6 is promoted to Productions..


In [98]:
archive_model(model_name, '5')

Model: RandomForestClassifier, Version: 5 is moved to Archive..


### model serving 모델 서빙
- Flask, FastAPI 형태로 REST API로 언제 만드냐,,
- MLFlow의 Model Serving 기능을 활용
- inference: input data를 주고 해당 값에 대한 예측을 return (API)

In [102]:
# model load & inference (version까지 관리되게)

model_name='RandomForestClassifier'
model_version='6'
model_uri=f'models:/{model_name}/{model_version}'    # model의 고유 ID

loaded_model = mlflow.pyfunc.load_model(model_uri)  
loaded_model

# 예측까지
test_input = X_test[:10]
loaded_model.predict(test_input)    # 보동 streamlit 연동해서 PM,PO,기획자한테 보여줌.


array([1, 2, 2, 1, 0, 1, 1, 0, 0, 1])

In [ ]:
# model을 요청했을때 그에 대한 응답값으로 API를 내려주는 방법

# mlflow 설치할때 Flask가 함께 설치되었다. 즉, 서버를 하나 더 운영해야한다.
# http://127.0.0.1:5000/#/experiments/841097937802182006/runs/6b36de56cd8a441fbc9951f372971230/artifacts/model

# 841097937802182006 : 실험 아이디
# 6b36de56cd8a441fbc9951f372971230: 실행 아이디

# API 서버 실행 명령어
mlflow models serve -m ./mlartifacts/841097937802182006/6b36de56cd8a441fbc9951f372971230/artifacts/model -p 5001 --no-conda

In [103]:
X_test_df = pd.DataFrame(X_test, columns=iris.feature_names)

In [116]:
X_test_df[:1].to_dict(orient='split')

{'index': [0],
 'columns': ['sepal length (cm)',
  'sepal width (cm)',
  'petal length (cm)',
  'petal width (cm)'],
 'data': [[0.5533332750260058,
   -1.282963310614858,
   0.6490834153533465,
   0.3957741007661703]]}

str

In [114]:
# requeat 모델 만들어서 post 형태로 요청하거나
# 아니면 thunder client 이용해도 된다

import requests

url = 'http://127.0.0.1:5001/invocations'
headers = {'Content-Type':'application/json'}
data = {
    'dataframe_split': X_test_df[:5].to_dict(orient='split')
}


<Response [200]>

In [ ]:

import json
type(json.dumps(data))  # python에서는 json 타입이 없다.


In [115]:

res = requests.post(url=url, headers=headers, data=json.dumps(data)) # payload에 data가 포함된 형태. get은 data가 없고, url 기반으로 소통
res.json()

{'predictions': [1, 2, 2, 1, 0]}

str